<a href="https://colab.research.google.com/github/fastdatascience/clinical_trial_risk/blob/fixes_nov_2022/train/ctgov/TrainCountryClassifier_16_Groups_LMIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import spacy
import pandas as pd
import re
import os
import sys
import pickle as pkl
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import json



In [8]:
!ls /content/drive/MyDrive/data

all_annotations.csv
arms_18_dev.spacy
arms_18_train.spacy
arms_20_dev.spacy
arms_20_train.spacy
countries_contexts.pkl.gz
country_16_train.spacy
country_model_01_keras.keras
dev_is_international_15.spacy
dev_is_international.spacy
dev_phase_04.spacy
dev.spacy
filtered_for_phase_arms_subjects_02.csv.bz2
filtered_for_phase_arms_subjects.csv.bz2
is_international_classifier.pkl.bz2
num_subjects_contexts.pkl.gz
num_subjects_model_05_keras.keras
num_subjects_model_06_keras.keras
num_subjects_model_10_keras.keras
num_subjects_positive_examples.pkl.gz
output_arms_18
output_arms_20
output_sap_01
output_subjects_11
output_subjects_12
output_subjects_13
output_textcat_05
output_textcat_06
output_textcat_07
output_textcat_is_international_11
output_textcat_is_international_15
output_textcat_phase_04
phase_arm_num_subjects_model_10_keras.keras
phase_arm_num_subjects_model_11_keras.keras
phase_model_05.keras
protocols.pkl.gz
protocols_small_test.pkl.gz
raw_protocols.zip
sap_01_dev.spacy
sap_01_train

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
lmics = {'AF',
                     'AL',
                     'AM',
                     'AO',
                     'AR',
                     'AS',
                     'AZ',
                     'BA',
                     'BD',
                     'BF',
                     'BG',
                     'BI',
                     'BJ',
                     'BO',
                     'BR',
                     'BT',
                     'BW',
                     'BY',
                     'BZ',
                     'CF',
                     'CI',
                     'CM',
                     'CN',
                     'CO',
                     'CR',
                     'CU',
                     'CV',
                     'DJ',
                     'DM',
                     'DO',
                     'DZ',
                     'EC',
                     'ER',
                     'ET',
                     'FJ',
                     'GA',
                     'GD',
                     'GE',
                     'GH',
                     'GN',
                     'GQ',
                     'GT',
                     'GW',
                     'GY',
                     'HN',
                     'HT',
                     'ID',
                     'IN',
                     'IQ',
                     'JM',
                     'JO',
                     'KE',
                     'KH',
                     'KI',
                     'KM',
                     'KZ',
                     'LB',
                     'LK',
                     'LR',
                     'LS',
                     'LY',
                     'MA',
                     'MD',
                     'ME',
                     'MG',
                     'MH',
                     'MK',
                     'ML',
                     'MM',
                     'MN',
                     'MR',
                     'MU',
                     'MV',
                     'MW',
                     'MX',
                     'MY',
                     'MZ',
                     'NA',
                     'NE',
                     'NG',
                     'NI',
                     'NP',
                     'PA',
                     'PE',
                     'PG',
                     'PH',
                     'PK',
                     'PY',
                     'RO',
                     'RS',
                     'RU',
                     'RW',
                     'SB',
                     'SD',
                     'SL',
                     'SN',
                     'SO',
                     'SR',
                     'SS',
                     'ST',
                     'SV',
                     'SY',
                     'SZ',
                     'TD',
                     'TG',
                     'TH',
                     'TJ',
                     'TL',
                     'TM',
                     'TN',
                     'TO',
                     'TR',
                     'TV',
                     'TZ',
                     'UA',
                     'UG',
                     'UZ',
                     'VN',
                     'VU',
                     'WS',
                     'ZA',
                     'ZM',
                     'ZW'}


In [ ]:
df_annotations = pd.read_csv("/content/drive/MyDrive/data/all_annotations.csv")


In [ ]:
df_annotations

,file,nct_id,train_val,num_arms,num_subjects,phase,intervention_type,country,has_sap
0,00_NCT00006400_Prot_SAP_000.pdf,NCT00006400,train,2.0,193.0,Phase 3,Drug,US,1
1,00_NCT00150800_Prot_000.pdf,NCT00150800,train,1.0,668.0,Phase 3,Drug,"US,AU,BR,CA,IN,MX",0
2,00_NCT00553800_Prot_SAP_000.pdf,NCT00553800,train,1.0,32.0,Phase 2,Drug,US,1
3,00_NCT00943800_Prot_SAP_000.pdf,NCT00943800,test,3.0,87.0,Not Applicable,Drug,US,1
4,00_NCT01030900_Prot_SAP_000.pdf,NCT01030900,train,1.0,50.0,Phase 2,BiologicalDrug,US,1
...,...,...,...,...,...,...,...,...,...
11919,99_NCT04405999_Prot_SAP_000.pdf,NCT04405999,train,2.0,50.0,Phase 4,Drug,RU,1
11920,99_NCT04490499_Prot_SAP_000.pdf,NCT04490499,train,1.0,207.0,Phase 3,Biological,FI,1
11921,99_NCT04529499_Prot_000.pdf,NCT04529499,train,2.0,353.0,Phase 3,Drug,KW,0
11922,99_NCT04945499_Prot_000.pdf,NCT04945499,test,1.0,100.0,Early Phase 1,Drug,CN,0


In [ ]:
df_annotations.country.value_counts().reset_index().iloc[:10]

,index,country
0,US,6222
1,JP,267
2,"US,CA",239
3,GB,196
4,CN,163
5,DE,111
6,CA,110
7,"US,PR",71
8,RU,56
9,IT,53


In [ ]:
def group_countries(countries):
  if pd.isna(countries):
    return None
  countries = set(countries.split(","))
  is_all_lmic = True
  is_some_lmic = False
  is_all_us_canada = True
  for country in countries:
    if country not in lmics:
      is_all_lmic = False
    if country in lmics:
      is_some_lmic = True
    if country not in {"US", "CA"}:
      is_all_us_canada = False
  if is_some_lmic:
    return "LMIC"
  if is_all_us_canada:
    return "USCA"
  return "HIGH_INCOME"

In [ ]:
df_annotations["clean_country"] = df_annotations.country.apply(group_countries)

In [ ]:
df_annotations["clean_country"].value_counts()

USCA           6573
HIGH_INCOME    2713
LMIC           2490
Name: clean_country, dtype: int64

# Get data

In [ ]:
with open("/content/drive/MyDrive/data/protocols.pkl.gz", "rb") as f:
    file_to_pages = pkl.load(f)

In [ ]:

texts = []

for i in range(len(df_annotations)):
    file_name = df_annotations.file.iloc[i]
    
    pages= file_to_pages[file_name]
    # Limit to first 3 pages because of memory constraints
    if len(pages) > 10:
      pages = pages[:10]

    texts.append(" ".join(pages))
df_annotations["text"] = texts

In [ ]:
df_annotations = df_annotations[~df_annotations.clean_country.isna()]

In [ ]:
labels = list(sorted(set(df_annotations["clean_country"])))

In [ ]:
df_annotations["clean_country"].value_counts()

USCA           6573
HIGH_INCOME    2713
LMIC           2490
Name: clean_country, dtype: int64

In [ ]:
import spacy
nlp = spacy.blank("en")
from spacy.tokens import DocBin

docs_train = []
for i in range(10):
  docs_train.append(DocBin())
docs_val = DocBin()

for idx in range(len(df_annotations)):
  if idx % 100 == 0:
    print (idx, len(df_annotations))
  if df_annotations.train_val.iloc[idx] == "train":
    docs_list = docs_train[idx % 10]
  else:
    docs_list = docs_val
  text = str(df_annotations.text.iloc[idx])
  if len(text) > 100000:
      text = text[:100000]
  doc = nlp(text)

  cats = {}
  for a in labels:
      cats[a] = 0
  if pd.isna(df_annotations.clean_country.iloc[idx]):
    continue
  cats[df_annotations.clean_country.iloc[idx]] = 1
  doc.cats = cats

  docs_list.add(doc)

0 11776
100 11776
200 11776
300 11776
400 11776
500 11776
600 11776
700 11776
800 11776
900 11776
1000 11776
1100 11776
1200 11776
1300 11776
1400 11776
1500 11776
1600 11776
1700 11776
1800 11776
1900 11776
2000 11776
2100 11776
2200 11776
2300 11776
2400 11776
2500 11776
2600 11776
2700 11776
2800 11776
2900 11776
3000 11776
3100 11776
3200 11776
3300 11776
3400 11776
3500 11776
3600 11776
3700 11776
3800 11776
3900 11776
4000 11776
4100 11776
4200 11776
4300 11776
4400 11776
4500 11776
4600 11776
4700 11776
4800 11776
4900 11776
5000 11776
5100 11776
5200 11776
5300 11776
5400 11776
5500 11776
5600 11776
5700 11776
5800 11776
5900 11776
6000 11776
6100 11776
6200 11776
6300 11776
6400 11776
6500 11776
6600 11776
6700 11776
6800 11776
6900 11776
7000 11776
7100 11776
7200 11776
7300 11776
7400 11776
7500 11776
7600 11776
7700 11776
7800 11776
7900 11776
8000 11776
8100 11776
8200 11776
8300 11776
8400 11776
8500 11776
8600 11776
8700 11776
8800 11776
8900 11776
9000 11776
9100 11776


In [ ]:
!mkdir train.spacy
!mkdir /content/drive/MyDrive/data/country_16_train.spacy


In [ ]:
for idx, doc_bin_train in enumerate(docs_train):
  doc_bin_train.to_disk(f"train.spacy/train_doc_bin{idx}.spacy")
  doc_bin_train.to_disk(f"/content/drive/MyDrive/data/country_16_train.spacy/train_doc_bin{idx}.spacy")

In [ ]:
!ls /content/drive/MyDrive/data/country_16_train.spacy

In [ ]:
docs_val.to_disk("dev.spacy")

In [ ]:
docs_val.to_disk("/content/drive/MyDrive/data/country_16_dev.spacy")

In [ ]:
note to self: running below line in TrainNumSubjectsClassifier_13_Spacy_Textcat_First10Pages_fewer_bins.ipynb

all_annotations.csv
arms_18_dev.spacy
arms_18_train.spacy
arms_20_dev.spacy
arms_20_train.spacy
countries_contexts.pkl.gz
country_16_train.spacy
country_model_01_keras.keras
dev_is_international_15.spacy
dev_is_international.spacy
dev_phase_04.spacy
dev.spacy
filtered_for_phase_arms_subjects_02.csv.bz2
filtered_for_phase_arms_subjects.csv.bz2
is_international_classifier.pkl.bz2
num_subjects_contexts.pkl.gz
num_subjects_model_05_keras.keras
num_subjects_model_06_keras.keras
num_subjects_model_10_keras.keras
num_subjects_positive_examples.pkl.gz
output_arms_18
output_arms_20
output_sap_01
output_subjects_11
output_subjects_12
output_subjects_13
output_textcat_05
output_textcat_06
output_textcat_07
output_textcat_is_international_11
output_textcat_is_international_15
output_textcat_phase_04
phase_arm_num_subjects_model_10_keras.keras
phase_arm_num_subjects_model_11_keras.keras
phase_model_05.keras
protocols.pkl.gz
protocols_small_test.pkl.gz
raw_protocols.zip
sap_01_dev.spacy
sap_01_train

In [ ]:
!python -m spacy train ./spacy_textcat_is_international_11.cfg --output /content/drive/MyDrive/data/output_country_16 --paths.train /content/drive/MyDrive/data/country_16_train.spacy --paths.dev  /content/drive/MyDrive/data/country_16_dev.spacy --gpu-id 0

✔ Created output directory:
/content/drive/MyDrive/data/output_sap_01
ℹ Saving to output directory:
/content/drive/MyDrive/data/output_sap_01
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-11-18 11:40:25,886] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-11-18 11:40:25,897] [INFO] Pipeline: ['textcat']
INFO:spacy:Pipeline: ['textcat']
[2022-11-18 11:40:25,901] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-11-18 11:40:25,901] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
tcmalloc: large alloc 1073750016 bytes == 0xb8c14000 @  0x7f59593452a4 0x58ead6 0x441ff3 0x5d1f81 0x5d2306 0x58f62c 0x5105e2 0x58fd37 0x50ca37 0x58fd37 0x50ca37 0x4d01b4 0x50cb8d 0x4d01b4 0x50cb8d 0x4d01b4 0x7f57edcea7c6 0x4ba72b 0x7f57edce8ce3 0x58f6e4 0x50ff13 0x5b575e 0x4bad0a 0x4d3249 0x591e56 0x50e18c 0x5b575e 0x58ff2e 0x50d482 0x5b575e 0x58ff2e
[2022-11-18 11:54:02,555] [

In [ ]:
import spacy
nlp = spacy.load("/content/drive/MyDrive/data/output_country_16/model-best")

In [ ]:
df_val = df_annotations[df_annotations.train_val=="val"]

predictions = []
for idx in range(len(df_val)):
    doc = nlp(df_val.text.apply(str).iloc[idx])
    predictions.append(doc.cats)

In [ ]:
# pred_proba = []
# for idx in range(len(df_val)):
#     pred_proba.append([predictions[idx][a] for a in range(num_classes)])

In [ ]:
predictions = []
for idx in range(len(df_val)):
    doc = nlp(df_val.text.apply(str).iloc[idx])
    predictions.append(doc.cats)

In [ ]:
df_val["y_pred"] = [max(p, key=p.get) for p in predictions]

In [ ]:
country_ground_truths = df_val.clean_country
pred_country = df_val["y_pred"]

In [ ]:
acc = accuracy_score(country_ground_truths, pred_country)
print (f"Country accuracy {acc}")

In [ ]:
ConfusionMatrixDisplay.from_predictions(country_ground_truths, pred_country, labels=labels)
plt.xticks(rotation=90)
;

In [ ]:
all_aucs = []
from sklearn.metrics import roc_auc_score
for i in labels:
  roc_auc = roc_auc_score(country_ground_truths == i, [p[i] for p in predictions])
  print (f"SAP ROC AUC {roc_auc} for {i} countries")
  all_aucs.append(roc_auc)
print ("Average", np.mean(all_aucs))

In [ ]:
plt.figure()
lw = 2
    
for i in labels:
    fpr, tpr, _ = roc_curve(country_ground_truths == i, [p[i] for p in predictions])
    roc_auc = auc(fpr, tpr)
    plt.plot(
        fpr,
        tpr,
        # color="darkorange",
        lw=lw,
        label=f"ROC curve {i} Country (area = {roc_auc:0.2f})"
    )
    plt.plot([0, 1], [0, 1], color="gray", alpha=0.5, lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic example")
plt.legend(loc="lower right")
plt.show()